In [57]:
import pandas as pd
import numpy as np
import sklearn.naive_bayes as NB
import sklearn.model_selection as cv
import sklearn.metrics as m
from sklearn import preprocessing
from statsmodels.stats.proportion import proportion_confint
import warnings
warnings.filterwarnings('ignore')

# Naive Bayes

## Read the data

As usual, before analyzing the data we read the csv and store all the values in a variable.

In [58]:
data = pd.read_csv('../datasets/preprocessed/train.csv', sep=',', na_values="NA")

In [59]:
data.head()

,MSSubClass,MSZoning,LotArea,LotShape,LandContour,LotConfig,Neighborhood,Condition1,BldgType,HouseStyle,...,MiscVal,SaleType,SaleCondition,SalePrice,MasVnr,SecondFloor,Baths,Porch,Pool,Id
0,G,RH,0.185945,1.0,Lvl,Inside,Edwards,Artery,1Fam,2Story,...,0.0,WD,Normal,Level2,0.0,1.0,0.4,True,0.0,0
1,A,RL,0.198890,1.0,Lvl,Inside,NAmes,Norm,1Fam,1Story,...,0.0,WD,Family,Level2,1.0,0.0,0.0,True,0.0,1
2,L,RL,0.260616,1.0,Lvl,Corner,NridgHt,Norm,Twnhs,1Story,...,0.0,New,Partial,Level4,1.0,0.0,0.4,True,0.0,2
3,A,RL,0.251230,1.0,Lvl,Inside,NAmes,Norm,1Fam,1Story,...,0.0,WD,Abnorml,Level1,1.0,0.0,0.0,False,0.0,3
4,E,RL,0.174186,1.0,Lvl,Inside,SWISU,Norm,1Fam,1.5Fin,...,0.0,WD,Normal,Level2,0.0,1.0,0.4,True,0.0,4


## Prepare the data

First, we begin by separating the data into two different variables: num_data, which only contains the numerical values, and cat_data, which only contains the categorical ones. We also exclude from cat_dat all the values that correspond to the column "SalePrice", since that's what we want to predict.

In [60]:
data
num_data = data.select_dtypes(include=np.number).drop(columns='Id')
cat_data = data.select_dtypes(include=['bool','object']).drop(columns='SalePrice')

## Train a model with numeric columns

First, we train a model using the numerical values in num_data. We're using Gaussian Naive Bayes which, as we can see, gives us a very poor score, only 0.206.

In [61]:
X = num_data
Y = data.loc[:,'SalePrice']

X_train, X_test, y_train, y_test = cv.train_test_split(X, Y, test_size=.3, random_state=1)

gnb = NB.GaussianNB()
gnb.fit(X_train,y_train)
pred=gnb.predict(X_test)
m.f1_score(y_test, pred, average='macro')

0.20642644731252324

## Train a model with categorical columns

Afterwards, we train a model with the categorical values in cat_data. Since we're using Multinomial Naive Bayes, which requires numerical tags instead of strings, we need to preprocess the categories in order to assign an integer ID to each different one before doing the training. As we can see, the final score is now 0.5, which is times better than the previous score, but still has room for improvement.

In [62]:
X = cat_data
Y = data.loc[:,'SalePrice']

data.dtypes
for col in X.columns:
    X.loc[:,col] = pd.factorize(X.loc[:,col])[0]
X.head()

X_train, X_test, y_train, y_test = cv.train_test_split(X, Y, test_size=.3, random_state=1)

mnb = NB.MultinomialNB()
mnb.fit(X_train,y_train)
pred=mnb.predict(X_test)
m.f1_score(y_test, pred, average='macro')

0.5001072522982637

## Cross validation of the best model

Now we do cross validation with the categorical values, as it's the best model for now, applying the same preprocessing we did for the previous model and Multinomial Naive Bayes again, but now calculating the cross_val_score. As we can see, the result is 0.427. We also build the confusion matrix and compute the f1 score, which again has almost the same value, and then we finish with the classification report.

In [63]:
kfold = cv.StratifiedKFold(n_splits=10, random_state=1) 

X = cat_data
Y = data.loc[:,'SalePrice']

for col in X.columns:
    X.loc[:,col] = pd.factorize(X.loc[:,col])[0]
X.head()

mnb = NB.MultinomialNB()

cvs = cv.cross_val_score(mnb,X=X,y=Y,cv=kfold,scoring='f1_macro')
np.mean(cvs)

0.4267319062102916

In [64]:
pred = cv.cross_val_predict(mnb, X=X, y=Y, cv=kfold)  

print(m.confusion_matrix(Y, pred))
print(m.f1_score(Y, pred, average='macro'))

[[ 46  32   1   0   0]
 [ 61 403 132  14  18]
 [  2  46 154  10  11]
 [  0   7  30   5  17]
 [  0   1   4   3   9]]
0.42740078633319734


In [65]:
print(m.classification_report(Y, pred))

              precision    recall  f1-score   support

      Level1       0.42      0.58      0.49        79
      Level2       0.82      0.64      0.72       628
      Level3       0.48      0.69      0.57       223
      Level4       0.16      0.08      0.11        59
      Level5       0.16      0.53      0.25        17

    accuracy                           0.61      1006
   macro avg       0.41      0.51      0.43      1006
weighted avg       0.67      0.61      0.63      1006



## Balancing the dataset

Despite having an accuracy of 0.613, the data of some of the categories of SalePrice was poorly classified. Level4, for example, has an f1-score as low as 0.11. This is due to having an unbalanced dataset, so we decide to balance our data so that the amount of instances for each SalePrice category is more or less the same.

In [66]:
Y.value_counts()

Level2    628
Level3    223
Level1     79
Level4     59
Level5     17
Name: SalePrice, dtype: int64

In [67]:
print(data['SalePrice'].unique())

X1 = data[data['SalePrice'] == 'Level1']
X2 = data[data['SalePrice'] == 'Level2']
X3 = data[data['SalePrice'] == 'Level3']
X4 = data[data['SalePrice'] == 'Level4']
X5 = data[data['SalePrice'] == 'Level5']

bdata = pd.DataFrame()

for i in range(3):
    bdata = bdata.append(X1, ignore_index = True)
bdata = bdata.append(X2.sample(frac=1/3), ignore_index = True)
bdata = bdata.append(X3, ignore_index = True)
for i in range(4):
    bdata = bdata.append(X4, ignore_index = True)
for i in range(10):
    bdata = bdata.append(X5, ignore_index = True)

bdata['SalePrice'].value_counts()

['Level2' 'Level4' 'Level1' 'Level3' 'Level5']


Level1    237
Level4    236
Level3    223
Level2    209
Level5    170
Name: SalePrice, dtype: int64

By balancing the dataset, we can see that the scores and accuracy are lower than before doing so (0.556), but the f1-score for the different levels is more consistent, with smaller diferences between them.

In [68]:
X = bdata.select_dtypes(include=['bool','object']).drop(columns=['SalePrice'])
Y = bdata['SalePrice']

kfold = cv.StratifiedKFold(n_splits=10) 

for col in X.columns:
    X.loc[:,col] = pd.factorize(X.loc[:,col])[0]
X.head()

mnb = NB.MultinomialNB()

cvs = cv.cross_val_score(mnb,X=X,y=Y,cv=kfold, scoring='f1_macro')
np.mean(cvs)

0.5670935699237525

In [69]:
pred = cv.cross_val_predict(mnb, X=X, y=Y, cv=kfold)  

print(m.confusion_matrix(Y, pred))
print(m.f1_score(Y, pred, average='macro'))

[[204  26   3   3   1]
 [ 52  88  51  14   4]
 [  5  43 100  57  18]
 [  0  19  51 112  54]
 [  0   0  20  38 112]]
0.5667520980998336


In [70]:
print(m.classification_report(Y, pred))

              precision    recall  f1-score   support

      Level1       0.78      0.86      0.82       237
      Level2       0.50      0.42      0.46       209
      Level3       0.44      0.45      0.45       223
      Level4       0.50      0.47      0.49       236
      Level5       0.59      0.66      0.62       170

    accuracy                           0.57      1075
   macro avg       0.56      0.57      0.57      1075
weighted avg       0.57      0.57      0.57      1075



## Naive Bayes using PCA 

Using PCA we obtain better results than before. We have a final score of 0.54 with Gaussian Naive Bayes and 0.6 for the cross validation. This is an improvement, but the numbers are still a little bit lower than desired.

In [71]:
data_pca = pd.read_csv('../datasets/preprocessed/train_pca.csv', sep=',', na_values="NA")

In [72]:
X = data_pca.drop(columns='Id')
Y = data.loc[:,'SalePrice']
X.head()

,0,1,2,3,4,5
0,-0.753748,0.636941,-0.164025,-0.204247,0.549835,0.539191
1,-0.117306,-0.688384,-0.493263,0.796882,-0.008604,-0.239237
2,0.787114,-0.469485,-0.626444,-0.272131,-0.467151,0.305142
3,-0.172825,-0.708999,-0.480148,0.864432,0.145394,-0.106748
4,-0.488369,0.642514,-0.179286,-0.202405,0.023691,-0.091447


In [73]:
X_train, X_test, y_train, y_test = cv.train_test_split(X, Y, test_size=.3, random_state=1)

gnb = NB.GaussianNB()
gnb.fit(X_train,y_train)
pred=gnb.predict(X_test)
m.f1_score(y_test, pred, average='macro')

0.5373145386766076

In [74]:
cvs = cv.cross_val_score(gnb,X=X,y=Y,cv=kfold,scoring='f1_macro')
np.mean(cvs)

0.5721330494202524

In [75]:
pred = cv.cross_val_predict(gnb, X=X, y=Y, cv=kfold)  

print(m.confusion_matrix(Y, pred))
print(m.f1_score(Y, pred, average='macro'))
print(m.classification_report(Y, pred))

[[ 35  44   0   0   0]
 [ 24 559  44   1   0]
 [  3  72 132  15   1]
 [  0   3  25  28   3]
 [  0   0   2   8   7]]
0.5953451439334121
              precision    recall  f1-score   support

      Level1       0.56      0.44      0.50        79
      Level2       0.82      0.89      0.86       628
      Level3       0.65      0.59      0.62       223
      Level4       0.54      0.47      0.50        59
      Level5       0.64      0.41      0.50        17

    accuracy                           0.76      1006
   macro avg       0.64      0.56      0.60      1006
weighted avg       0.75      0.76      0.75      1006



## Testing with test dataset

In [76]:
data_test_pca = pd.read_csv('../datasets/preprocessed/test_pca.csv', sep=',')
data_test = pd.read_csv('../datasets/preprocessed/test.csv', sep=',')
y_test = data_test['SalePrice']
data_test_pca.drop(columns='Id',inplace=True)

In [77]:
pred = cv.cross_val_predict(gnb, X=data_test_pca, y=y_test, cv=kfold)  
print(m.classification_report(y_test, pred))
accuracy = m.accuracy_score(y_test, pred)
precision = m.precision_score(y_test, pred, average='macro')
recall = m.recall_score(y_test, pred, average='macro')
f1 = m.f1_score(y_test, pred, average='macro')
print('F1 macro: ' + str(f1) + ', Accuracy: ' + str(accuracy) + ', Precision macro: ' + str(precision) + ', Recall macro: ' + str(recall))

              precision    recall  f1-score   support

      Level1       0.40      0.34      0.37        35
      Level2       0.82      0.88      0.85       276
      Level3       0.67      0.60      0.63        88
      Level4       0.53      0.36      0.43        25
      Level5       0.62      0.62      0.62         8

    accuracy                           0.75       432
   macro avg       0.61      0.56      0.58       432
weighted avg       0.73      0.75      0.74       432

F1 macro: 0.5808445264389472, Accuracy: 0.7453703703703703, Precision macro: 0.6081468164531976, Recall macro: 0.5621129305477132


In [78]:
epsilon = m.f1_score(y_test, pred, average='macro')
print("Confidence interval: ",proportion_confint(count=epsilon*data_test_pca.shape[0], nobs=data_test_pca.shape[0], alpha=0.05, method='binom_test'))

Confidence interval:  (0.5312718255649439, 0.625091645922958)
